In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 设置中文字体
matplotlib.rcParams['font.family'] = 'SimHei'
matplotlib.rcParams['axes.unicode_minus'] = False

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义数据集类（保持不变）
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, transform=None, partition=0):
        self.img_dir = img_dir
        self.transform = transform

        attr_df = pd.read_csv(attr_path, sep=',', header=0)
        partition_df = pd.read_csv(partition_path, sep=',', header=0)
        attr_df = attr_df.merge(partition_df, on='image_id')
        self.attr_df = attr_df[attr_df['partition'] == partition]
        bbox_df = pd.read_csv(bbox_path, sep=',', header=0)
        self.attr_df = self.attr_df.merge(bbox_df, on='image_id')

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        img_name = self.attr_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        attrs = self.attr_df.iloc[idx, 1:41].values
        attrs = (attrs + 1) // 2
        attrs = attrs.astype(np.float32)
        if self.transform:
            image = self.transform(image)
        return image, attrs

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 数据集路径设置（请根据实际路径修改）
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'

# 创建训练集和验证集
train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=1)

# 数据加载器
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 定义CVAE模型（保持不变）
class CVAE(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, cond_dim=40):
        super(CVAE, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c):
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c):
        z = torch.cat([z, c], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c)
        return recon_x, mu, logvar

# 定义损失函数（保持不变）
def loss_function(recon_x, x, mu, logvar):
    recon_loss = nn.MSELoss(reduction='sum')(recon_x, x)
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KL

# 封装训练函数
def train_cvae(lr, num_epochs, train_loader, val_loader, device, latent_dim=128, cond_dim=40):
    model = CVAE(img_channels=3, img_size=64, latent_dim=latent_dim, cond_dim=cond_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='sum')
    
    train_losses = []
    val_losses = []
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0
        for batch_idx, (data, attrs) in enumerate(tqdm(train_loader, desc=f"学习率 {lr} - 训练 Epoch {epoch}/{num_epochs}")):
            data = data.to(device)
            attrs = attrs.to(device)
            
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data, attrs)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        
        avg_train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)
        print(f"学习率 {lr}，第 {epoch} 轮，训练集平均损失: {avg_train_loss:.4f}")
        
        # 验证集评估
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, attrs in val_loader:
                data = data.to(device)
                attrs = attrs.to(device)
                recon_batch, mu, logvar = model(data, attrs)
                loss = loss_function(recon_batch, data, mu, logvar)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)
        print(f"学习率 {lr}，第 {epoch} 轮，验证集平均损失: {avg_val_loss:.4f}")
    
    return train_losses, val_losses

# 定义要测试的学习率列表
learning_rates = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]

# 定义训练参数
num_epochs = 20

# 创建一个字典来存储不同学习率的损失记录
loss_history = {
    '学习率': learning_rates,
    '训练损失': [],
    '验证损失': []
}

# 对每个学习率进行训练
for lr in learning_rates:
    train_loss, val_loss = train_cvae(lr, num_epochs, train_loader, val_loader, device)
    loss_history['训练损失'].append(train_loss)
    loss_history['验证损失'].append(val_loss)

# 绘制训练损失曲线
plt.figure(figsize=(12, 6))
for idx, lr in enumerate(learning_rates):
    plt.plot(range(1, num_epochs + 1), loss_history['训练损失'][idx], label=f'训练 LR={lr}')
plt.title("不同学习率下的训练损失")
plt.xlabel("Epoch")
plt.ylabel("平均损失")
plt.legend()
plt.grid(True)
plt.show()

# 绘制验证损失曲线
plt.figure(figsize=(12, 6))
for idx, lr in enumerate(learning_rates):
    plt.plot(range(1, num_epochs + 1), loss_history['验证损失'][idx], label=f'验证 LR={lr}')
plt.title("不同学习率下的验证损失")
plt.xlabel("Epoch")
plt.ylabel("平均损失")
plt.legend()
plt.grid(True)
plt.show()

# 保存模型示例（可选）
# 您可以选择保存不同学习率下表现最好的模型
# torch.save(model.state_dict(), f'cvae_celeba_lr_{lr}.pth')
# print(f"模型已保存为 cvae_celeba_lr_{lr}.pth")


### 评估不同学习率 (learning rate) 对模型训练过程中平均损失的影响，并确保图表中的标题和标签能够正常显示中文

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 设置中文字体
matplotlib.rcParams['font.family'] = 'SimHei'
matplotlib.rcParams['axes.unicode_minus'] = False

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义数据集类（保持不变）
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, transform=None, partition=0):
        self.img_dir = img_dir
        self.transform = transform

        attr_df = pd.read_csv(attr_path, sep=',', header=0)
        partition_df = pd.read_csv(partition_path, sep=',', header=0)
        attr_df = attr_df.merge(partition_df, on='image_id')
        self.attr_df = attr_df[attr_df['partition'] == partition]
        bbox_df = pd.read_csv(bbox_path, sep=',', header=0)
        self.attr_df = self.attr_df.merge(bbox_df, on='image_id')

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        img_name = self.attr_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        attrs = self.attr_df.iloc[idx, 1:41].values
        attrs = (attrs + 1) // 2
        attrs = attrs.astype(np.float32)
        if self.transform:
            image = self.transform(image)
        return image, attrs

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 数据集路径设置（请根据实际路径修改）
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'

# 创建训练集和验证集
train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=0)
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=1)

# 数据加载器
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 定义CVAE模型（保持不变）
class CVAE(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, cond_dim=40):
        super(CVAE, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c):
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c):
        z = torch.cat([z, c], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c)
        return recon_x, mu, logvar

# 定义损失函数（保持不变）
def loss_function(recon_x, x, mu, logvar):
    recon_loss = nn.MSELoss(reduction='sum')(recon_x, x)
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KL

# 封装训练函数
def train_cvae(lr, num_epochs, train_loader, val_loader, device, latent_dim=128, cond_dim=40):
    model = CVAE(img_channels=3, img_size=64, latent_dim=latent_dim, cond_dim=cond_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='sum')
    
    train_losses = []
    val_losses = []
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0
        for batch_idx, (data, attrs) in enumerate(tqdm(train_loader, desc=f"学习率 {lr} - 训练 Epoch {epoch}/{num_epochs}")):
            data = data.to(device)
            attrs = attrs.to(device)
            
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data, attrs)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        
        avg_train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)
        print(f"学习率 {lr}，第 {epoch} 轮，训练集平均损失: {avg_train_loss:.4f}")
        
        # 验证集评估
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, attrs in val_loader:
                data = data.to(device)
                attrs = attrs.to(device)
                recon_batch, mu, logvar = model(data, attrs)
                loss = loss_function(recon_batch, data, mu, logvar)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)
        print(f"学习率 {lr}，第 {epoch} 轮，验证集平均损失: {avg_val_loss:.4f}")
    
    return train_losses, val_losses

# 定义要测试的学习率列表
learning_rates = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]

# 定义训练参数
num_epochs = 20

# 创建一个字典来存储不同学习率的损失记录
loss_history = {
    '学习率': learning_rates,
    '训练损失': [],
    '验证损失': []
}

# 对每个学习率进行训练
for lr in learning_rates:
    train_loss, val_loss = train_cvae(lr, num_epochs, train_loader, val_loader, device)
    loss_history['训练损失'].append(train_loss)
    loss_history['验证损失'].append(val_loss)

# 绘制训练损失曲线
plt.figure(figsize=(12, 6))
for idx, lr in enumerate(learning_rates):
    plt.plot(range(1, num_epochs + 1), loss_history['训练损失'][idx], label=f'训练 LR={lr}')
plt.title("不同学习率下的训练损失")
plt.xlabel("Epoch")
plt.ylabel("平均损失")
plt.legend()
plt.grid(True)
plt.show()

# 绘制验证损失曲线
plt.figure(figsize=(12, 6))
for idx, lr in enumerate(learning_rates):
    plt.plot(range(1, num_epochs + 1), loss_history['验证损失'][idx], label=f'验证 LR={lr}')
plt.title("不同学习率下的验证损失")
plt.xlabel("Epoch")
plt.ylabel("平均损失")
plt.legend()
plt.grid(True)
plt.show()

# 保存模型示例（可选）
# 您可以选择保存不同学习率下表现最好的模型
# torch.save(model.state_dict(), f'cvae_celeba_lr_{lr}.pth')
# print(f"模型已保存为 cvae_celeba_lr_{lr}.pth")


使用设备: cuda


学习率 0.0001 - 训练 Epoch 1/20: 100%|██████████| 1272/1272 [01:06<00:00, 19.02it/s]

学习率 0.0001，第 1 轮，训练集平均损失: 1004.1189


学习率 0.0001，第 1 轮，验证集平均损失: 645.4876


学习率 0.0001 - 训练 Epoch 2/20: 100%|██████████| 1272/1272 [01:02<00:00, 20.44it/s]

...

学习率 0.0005 - 训练 Epoch 17/20: 100%|██████████| 1272/1272 [01:01<00:00, 20.71it/s]

学习率 0.0005，第 17 轮，训练集平均损失: 426.8011


学习率 0.0005，第 17 轮，验证集平均损失: 430.2249


学习率 0.0005 - 训练 Epoch 18/20:  34%|███▍      | 433/1272 [00:23<00:44, 18.76it/s]


KeyboardInterrupt: 